In [ ]:
# 텍스트 분류용 Transformer 모델 (Sentiment Analysis, 긍정/부정 분류)
# Naver 데이터셋 : 다국어 영화 리뷰(긍정/부정)

In [12]:
# 1. 라이브러리 설치 및 임포트
import torch
# from transformers import BertTokenizer, BertForSequenceClassification
# from transformers import DistilBertTokenizer, DistilBertForSequenceClassification # 경량 모델
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

In [13]:
# 2. 데이터셋 로드

# CSV 파일 직접 로드 (train/test 분리된 버전)
dataset = load_dataset(
    "csv", 
    data_files={
        "train": "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt",
        "test": "https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt"
    }, 
    delimiter="\t"
)

# 빠른 테스트용: 데이터 크기 줄이기(학습용 2000개, 테스트용 500개만 사용)
small_train = dataset['train'].shuffle(seed=42).select(range(10000))
small_test = dataset['test'].shuffle(seed=42).select(range(2000))
# print(small_train[0])
# print(small_test[0])

# 최종 학습용: 전체 20만 건 사용
full_train = dataset["train"]
full_test = dataset["test"]

In [14]:
# 3. 토큰화

# BERT의 사전학습된 토크나이저 모델 로드, 
# bert-base-uncased는 영어 BERT 모델로, 모든 단어를 소문자(uncased)롤 변환
# - 토크나이저는 문장을 단어 → 토큰 → 숫자 ID로 바꿔주는 역할을 합니다. 예: "I love movies" → [101, 1045, 2293, 3185, 102]
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# 경량 모델 DistilBERT 토크나이저 준비
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base') # 다국어 모델 적용

def tokenize(batch):
    texts = [str(x) for x in batch['document']]
    # batch['text'] 데이터셋에서 텍스트 분분을 가져온다
    # padding='max_length' 모든 문장을 동일한 길이로 맞추가 위해 패딩(padding)을 추가(짧은 문장은 [PAD] 토큰으로 채움)
    # truncation=True 너무 긴 문장은 잘라내기(truncate)를 해서 모델 입력 크기에 맞춘다
    # 각 문장이 토큰 ID, attention mask 등으로 변환된 딕셔너리 형태로 반환
    return tokenizer(texts, padding='max_length', truncation=True, max_length=128) # 경량 모델

# 결측치 제거 후 토큰화
small_train = small_train.filter(lambda x: x["document"] is not None and x["document"] != "")
small_test = small_test.filter(lambda x: x["document"] is not None and x["document"] != "")
full_train = full_train.filter(lambda x: x["document"] is not None and x["document"] != "")
full_test = full_test.filter(lambda x: x["document"] is not None and x["document"] != "")

# dataset.map() 데이터셋 모든 샘플에 tokenize 함수를 적용
# batched=True 여러 샘플을 한번에 묶어서 처리 -> 속도 향상
# 결과적으로 원래 텍스트 데이터셋에 토큰화된 입력값(input_ids, attention_mask)이 추가
# dataset = dataset.map(tokenize, batched=True)

small_train = small_train.map(tokenize, batched=True) # 다국어 모델
small_test = small_test.map(tokenize, batched=True)
full_train = full_train.map(tokenize, batched=True) # 다국어 모델
full_test = full_test.map(tokenize, batched=True)

# 라벨 컬럼 이름 변경 및 불필요한 컬럼 제거
def prepare_dataset(ds):
    ds = ds.rename_column("label", "labels")
    ds = ds.remove_columns(["id", "document"])
    ds.set_format("torch")
    return ds

small_train = prepare_dataset(small_train)
small_test = prepare_dataset(small_test)
full_train = prepare_dataset(full_train)
full_test = prepare_dataset(full_test)


Map: 100%|██████████| 2000/2000 [00:00<00:00, 16529.02 examples/s]


In [15]:
# 4. 모델 불러오기

# HuggingFace transformers 라이브러리에서 제공하는 BERT 기반 텍스트 분류 모델 클래스
# 기존 BERT 모델 위에 분류용 헤드(classification head)가 추가된 구조
# - BERT 본체: 입력 문장을 토큰 단위로 인코딩 -> 문맥적 벡터 표현 생성
# - Classification Head: [CLS] 토큰의 벡터를 받아 Dense Layer + Softmax로 분류 결과 출력

# from_pretrained("bert-base-uncased") 사전학습된 bert-base-uncased 모델 로드, uncased -> 모든 영어 단어를 소문자로 변환해서 처리
# 이미 대규모 코퍼스(위키백과, BookCorpus 등)로 학습된 모델을 가져오기 때문에, 처음부터 학습하지 않고도 좋은 성능을 낼 수 있다
# num_labels=2 분류할 클래스 개수를 지정 -> 여기서는 2개 클래스 예시) 긍정(Positive)/부정(Negative), 뉴스 카테고리 분류 num_labels=4(정치,경제,스포츠,기술)

# model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = AutoModelForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=2) # 다국어 분류 모델

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# 5. 학습 파라미터 설정
trainer_args = TrainingArguments(
    output_dir='./results_sentiment_analysis_naver_xlm-roberta-base', # 학습된 모델과 체크포인트(중간 저장 파일)를 저장할 디렉토리 경로, 학습 도중과 완료 후 모델 가중치가 이 폴더에 저장
    eval_strategy='epoch', # 매 epoch(한번 전체 데이터셋을 학습)마다 평가를 수행
    save_strategy='epoch',
    logging_strategy='steps',
    logging_steps=50,    
    per_device_train_batch_size=16, # 배치 사이즈 16
    num_train_epochs=3, # 학습 반복 3회, 전체 데이터셋을 3회 학습
    learning_rate=3e-5,
    weight_decay=0.01,
    logging_dir='./logs', # 학습 과정의 로그를 저장
    fp16=True, # 혼합 정밀도 -> 속도 향상
)

In [17]:
# 6. Trainer 구성

# Trainer 객체 생성, Hugging Face에서 제공하는 고수준 학습 관리 클래스, 모델 학습/평가/저장 자동 처리
# PyTorch의 DataLoader, 학습 루프, 옵티마이저 등을 직접 작성하지 않아도 됨
trainer = Trainer(
    model=model, # 학습 모델 지정(BertForSequenceClassification), BERT + 분류 헤드 구조가 학습 대상
    args=trainer_args, # 학습 파라미터 전달
    train_dataset=small_train, # 학습 데이터셋 지정, 토큰화된 입력(input_ids, attention_mask)과 라벨(label)이 포함
    eval_dataset=small_test, # 평가 데이터셋 지정, 학습 중간이나 학습 후 성능을 측정할때 사용
)

# 최종 학습용 학습 (전체 데이터셋)
# trainer = Trainer(
#     model=model,
#     args=training_args,
#     train_dataset=full_train,
#     eval_dataset=full_test,
# )


In [18]:
# 7. 학습 실행
trainer.train()

Epoch,Training Loss,Validation Loss
1,0.448000,0.492413
2,0.368100,0.436799
3,0.288100,0.424528


TrainOutput(global_step=1875, training_loss=0.42062521514892576, metrics={'train_runtime': 6544.5615, 'train_samples_per_second': 4.584, 'train_steps_per_second': 0.286, 'total_flos': 1973332915200000.0, 'train_loss': 0.42062521514892576, 'epoch': 3.0})

In [9]:
# 8. 모델 평가
results = trainer.evaluate()
print(results)

{'eval_loss': 0.46613526344299316, 'eval_runtime': 10.6138, 'eval_samples_per_second': 94.217, 'eval_steps_per_second': 11.777, 'epoch': 3.0}


In [22]:
# 9. 추론 테스트
# test_sentences = [
#     "I really loved this movie, it was fantastic!",
#     "The film was boring and too long.",
#     "An average movie, not too bad but not great either.",
#     "The acting was brilliant and kept me engaged.",
#     "I fell asleep halfway through, it was so dull.",
#     "The storyline was unique and very exciting.",
#     "The dialogue felt forced and unnatural.",
#     "I enjoyed every moment, truly a masterpiece!",
#     "The special effects were cheap and disappointing.",
#     "It was okay, but I wouldn’t watch it again."
# ]

test_sentences = [
    "이 영화 정말 재미있었어요!",
    "스토리가 지루하고 너무 길었어요.",
    "그럭저럭 볼만했지만 특별한 건 없었어요.",
    "배우들의 연기가 훌륭해서 몰입할 수 있었습니다.",
    "중간에 잠들 정도로 지루했어요.",
    "전개가 신선하고 흥미로웠습니다.",
    "대사가 어색하고 자연스럽지 않았습니다.",
    "처음부터 끝까지 즐겁게 볼 수 있었어요!",
    "특수효과가 싸구려 같아서 실망했어요.",
    "괜찮았지만 다시 보고 싶지는 않네요."
]

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# 토큰화
inputs = tokenizer(
    test_sentences, # 추론 테스트 문장
    padding=True, # 문장 길이를 맞추기 위해 패딩 추가
    truncation=True, # 너무 긴 문장은 잘라냄
    max_length=128, # 최대 토큰 길이 제한
    return_tensors='pt' # PyTorch 텐서로 반환
).to(device) # 입력 텐서를 GPU/CPU로 이동

# 모델 예측

# **inputs -> 파이썬 언패킹 문법을 사용해 딕셔너리의 키-값을 함수 인자로 전달
# 내부적으로 호출 -> model(input_ids=..., attention_mask=...)
outputs = model(**inputs)

# outputs.logits 각 클래스(긍정/부정)에 대한 점수, 예시) [[-1.3135,  1.1777],[ 1.1172, -1.1963],[ 0.2496, -0.2385]]
# 가장 높은 점수를 가진 클래스 선택 -> 결과 [1,0,0] (0=부정, 1=긍정), 예시) [1, 0, 0]
predictions = torch.argmax(outputs.logits, dim=1)
print('예측 결과:', predictions)

# 확률값 계산
probs = torch.softmax(outputs.logits, dim=1)

# test_sentences -> 추론에 테스트 할 문장
# predictions -> 모델이 예측한 결과(0 또는 1 값이 들어 있는 텐서)
# zip(test_sentences, predictions) -> 두 리스트/텐서를 묶어서 문장과 예측값을 한쌍으로 반복할 수 있게 만든다
for i, (sentence, pred) in enumerate(zip(test_sentences, predictions)):
    label = '긍정' if pred.item() == 1 else '부정'
    confidence = probs[i, pred.item()].item()  # 행(i), 열(pred) 지정
    print(f'문장: {sentence}\n예측: {label} (확률: {confidence:.2f})\n')


예측 결과: tensor([1, 0, 0, 1, 0, 1, 0, 1, 0, 1], device='cuda:0')
문장: 이 영화 정말 재미있었어요!
예측: 긍정 (확률: 0.98)

문장: 스토리가 지루하고 너무 길었어요.
예측: 부정 (확률: 0.99)

문장: 그럭저럭 볼만했지만 특별한 건 없었어요.
예측: 부정 (확률: 0.97)

문장: 배우들의 연기가 훌륭해서 몰입할 수 있었습니다.
예측: 긍정 (확률: 0.98)

문장: 중간에 잠들 정도로 지루했어요.
예측: 부정 (확률: 0.99)

문장: 전개가 신선하고 흥미로웠습니다.
예측: 긍정 (확률: 0.98)

문장: 대사가 어색하고 자연스럽지 않았습니다.
예측: 부정 (확률: 0.91)

문장: 처음부터 끝까지 즐겁게 볼 수 있었어요!
예측: 긍정 (확률: 0.99)

문장: 특수효과가 싸구려 같아서 실망했어요.
예측: 부정 (확률: 0.98)

문장: 괜찮았지만 다시 보고 싶지는 않네요.
예측: 긍정 (확률: 0.99)

